In [67]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
import cv2
import mediapipe as mp
import os
import subprocess
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Prepare Data Testing Image 

In [68]:
def list_jpg_png_files(directory):
    jpg_png_files = []
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith('.jpg') or filename.endswith('.png'):
                file_path = os.path.join(root, filename)
                file_path = file_path.replace('\\', '/')
                jpg_png_files.append(file_path)
    return jpg_png_files

list_jpg_png_files('./datasets/Testing-Image')

['./datasets/Testing-Image/0_Apa_1.jpg',
 './datasets/Testing-Image/10_Apa_1.jpg',
 './datasets/Testing-Image/11_Apa_1.jpg',
 './datasets/Testing-Image/12_Apa_1.jpg',
 './datasets/Testing-Image/13_Apa_1.jpg',
 './datasets/Testing-Image/14_Apa_1.jpg',
 './datasets/Testing-Image/15_Apa_1.jpg',
 './datasets/Testing-Image/16_Apa_1.jpg',
 './datasets/Testing-Image/17_Apa_1.jpg',
 './datasets/Testing-Image/18_Apa_1.jpg',
 './datasets/Testing-Image/1_Apa_1.jpg',
 './datasets/Testing-Image/2_Apa_1.jpg',
 './datasets/Testing-Image/3_Apa_1.jpg',
 './datasets/Testing-Image/4_Apa_1.jpg',
 './datasets/Testing-Image/5_Apa_1.jpg',
 './datasets/Testing-Image/6_Apa_1.jpg',
 './datasets/Testing-Image/7_Apa_1.jpg',
 './datasets/Testing-Image/8_Apa_1.jpg',
 './datasets/Testing-Image/9_Apa_1.jpg',
 './datasets/Testing-Image/A_0.jpg',
 './datasets/Testing-Image/A_1.jpg',
 './datasets/Testing-Image/A_10.jpg',
 './datasets/Testing-Image/A_11.jpg',
 './datasets/Testing-Image/A_12.jpg',
 './datasets/Testing-Ima

# Test Static Image

In [126]:
def extract_hand_features(image_path):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5
    )

    data = []

    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for landmark in hand_landmarks.landmark:
                landmarks.append(landmark.x)
                landmarks.append(landmark.y)
                landmarks.append(landmark.z)
            data.append([os.path.basename(image_path)] + landmarks)

    hands.close()

    return data


# sample_data = extract_hand_features("datasets/Testing-Image/0_Apa_1.jpg")[0][1:]
samples = []
for file in list_jpg_png_files('./datasets/Testing-Image'):
    samples += extract_hand_features(file)

c:\Users\ahmad\AppData\Local\Programs\Python\Python39\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '



# Test Predict

In [127]:
import joblib
import numpy as np

scaler = joblib.load("scaler.pkl")

for i in range(len(samples)):
    # # with scaler 
    samples[i] = samples[i][1:]
    samples[i] = np.array(samples[i]).reshape(1, -1)
    samples[i] = scaler.transform(samples[i])

    # without scaler
    # samples[i] =  np.expand_dims(samples[i][1:], axis=0)

label_encoder = joblib.load("label_encoder.pkl")

c:\Users\ahmad\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ahmad\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ahmad\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ahmad\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ahmad\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler wa

In [128]:
import tensorflow as tf

model_path = 'models/14-juni-1058am.h5'
model = tf.keras.models.load_model(model_path)
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 64)                4096      
                                                                 
 dense_34 (Dense)            (None, 32)                2080      
                                                                 
 dense_35 (Dense)            (None, 66)                2178      
                                                                 
Total params: 8354 (32.63 KB)
Trainable params: 8354 (32.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [129]:
import numpy as np

count = 0
loss = 0
for i in range(len(samples)):
    prediction = model.predict(samples[i])
    prediction = np.argmax(prediction, axis=1)
    class_labels = ['A', 'Aku', 'Apa', 'B', 'Bagaimana', 'Baik', 'Bapak', 'Berapa',
       'Besok', 'C', 'D', 'Dia', 'Dimana', 'E', 'F', 'G', 'H', 'Halo',
       'Hari ini', 'I', 'Ibu', 'J', 'Jawab', 'K', 'Kalian', 'Kamu',
       'Kantor', 'Kapan', 'Kemana', 'Kemarin', 'Kerja', 'L', 'Lelah',
       'Lusa', 'M', 'Maaf', 'Makan', 'Malam', 'Mengapa', 'N', 'Nanti',
       'O', 'P', 'Pagi', 'Q', 'R', 'S', 'Sabar', 'Sakit', 'Sama - sama',
       'Sedih', 'Sekarang', 'Senang', 'Siang', 'Siapa', 'Sore', 'T',
       'Terima kasih', 'Tidur', 'Tolong', 'U', 'V', 'W', 'X', 'Y', 'Z']
    prediction = class_labels[prediction[0]]
    if prediction in ['Apa', 'A', 'Q', 'Baik']:
        count += 1
    else:
        loss += 1
    print(f"{prediction}")

print(f"Count: {count}")
print(f"Loss: {loss}")

1/1 [==============================] - 0s 112ms/step
Apa
1/1 [==============================] - 0s 38ms/step
Baik
1/1 [==============================] - 0s 37ms/step
Baik
1/1 [==============================] - 0s 33ms/step
Senang
1/1 [==============================] - 0s 30ms/step
Senang
1/1 [==============================] - 0s 24ms/step
Senang
1/1 [==============================] - 0s 34ms/step
Senang
1/1 [==============================] - 0s 27ms/step
Baik
1/1 [==============================] - 0s 28ms/step
Baik
1/1 [==============================] - 0s 25ms/step
Baik
1/1 [==============================] - 0s 26ms/step
Baik
1/1 [==============================] - 0s 29ms/step
Baik
1/1 [==============================] - 0s 19ms/step
Baik
1/1 [==============================] - 0s 21ms/step
Baik
1/1 [==============================] - 0s 21ms/step
Baik
1/1 [==============================] - 0s 17ms/step
Baik
1/1 [==============================] - 0s 17ms/step
Apa
1/1 [===================

In [102]:
predicted_class_index = np.argmax(prediction)
print(predicted_class_index)

0


In [77]:
class_labels = ['A', 'Aku', 'Apa', 'B', 'Bagaimana', 'Baik', 'Bapak', 'Berapa',
       'Besok', 'C', 'D', 'Dia', 'Dimana', 'E', 'F', 'G', 'H', 'Halo',
       'Hari ini', 'I', 'Ibu', 'J', 'Jawab', 'K', 'Kalian', 'Kamu',
       'Kantor', 'Kapan', 'Kemana', 'Kemarin', 'Kerja', 'L', 'Lelah',
       'Lusa', 'M', 'Maaf', 'Makan', 'Malam', 'Mengapa', 'N', 'Nanti',
       'O', 'P', 'Pagi', 'Q', 'R', 'S', 'Sabar', 'Sakit', 'Sama - sama',
       'Sedih', 'Sekarang', 'Senang', 'Siang', 'Siapa', 'Sore', 'T',
       'Terima kasih', 'Tidur', 'Tolong', 'U', 'V', 'W', 'X', 'Y', 'Z']


predicted_class = class_labels[predicted_class_index]
predicted_class

'Apa'

# DIEEEEEEEEEEEEEEE

# Function For Testing Directory